<a href="https://colab.research.google.com/github/Phillistan16/imoco_recon/blob/colab_demo/colab-imoco-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# iterative Motion Compensation (iMoCo) Reconstruction Demo
This CoLab notebook is a demo of the iMoCo method
https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.27998
https://github.com/PulmonaryMRI/imoco_recon


# Setup

<!-- By default, this colab notebook should have a GPU runtime environment. To check this, go to **Runtime > Change runtime type** and see if **GPU** is selected as **Hardware accelerator**. -->

To setup, we will install [SigPy](http://sigpy.readthedocs.io), [ANTsPy](https://github.com/ANTsX/ANTsPy), and clone the iMoCo github repo.

In [1]:
# install dependent packages sigpy, h5py, antspy
!pip install sigpy h5py
!pip install antspyx

# clone imoco recon repo
! git clone -b luis_test https://github.com/PulmonaryMRI/imoco_recon

     |████████████████████████████████| 133kB 4.7MB/s 
     |████████████████████████████████| 254.6MB 58kB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
Cloning into 'imoco_recon'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 126 (delta 40), reused 108 (delta 30), pack-reused 0
Receiving objects: 100% (126/126), 59.17 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (40/40), done.


# Download dataset

Now, let's download a sample Lung UTE dataset. The dataset is hosted on Zenodo: https://zenodo.org/record/3733776#.XoJlNC2ZPOQ.

<!-- The `download_lung_dataset()` function will download the datasets if it cannot find them under `data/lung/`. It takes around 8 minutes to download the datasets. -->

In [1]:
!mkdir lung_mri
!pip install zenodo-get
!zenodo_get -d 10.5281/zenodo.3733775 -o ./lung_mri

mkdir: cannot create directory ‘lung_mri’: File exists
Title: Free Breathing Lung MRI Dataset at 3T
Keywords: lung MRI, motion correction
Publication date: 2020-03-30
DOI: 10.5281/zenodo.3733776
Total size: 4800.4 MB

Link: https://zenodo.org/api/files/4e2fa077-5616-489c-b70f-3464b7f2ec44/cfl.py   size: 0.0 MB

Checksum is correct. (54d868ad54b8a2b35606d5771f53c0e0)

Link: https://zenodo.org/api/files/4e2fa077-5616-489c-b70f-3464b7f2ec44/MRI_Raw_datam.cfl   size: 3074.7 MB

Checksum is correct. (ec2e8d5f885147873ce386d99dc90343)

Link: https://zenodo.org/api/files/4e2fa077-5616-489c-b70f-3464b7f2ec44/MRI_Raw_datam.hdr   size: 0.0 MB

Checksum is correct. (f2bf52eb84dd2bc8299c8f058306b765)

Link: https://zenodo.org/api/files/4e2fa077-5616-489c-b70f-3464b7f2ec44/MRI_Raw_dcf2m.cfl   size: 384.3 MB

Checksum is correct. (8de658fd47d671d0c2aa43b191920ea3)

Link: https://zenodo.org/api/files/4e2fa077-5616-489c-b70f-3464b7f2ec44/MRI_Raw_dcf2m.hdr   size: 0.0 MB

Checksum is correct. (e5b9482e

# Crop Data (Optional)
decrease number of frequency encoding points to adapt for Colab RAM

In [1]:
import imoco_recon.imoco_py.sigpy_e.cfl as cfl
import numpy as np
num_ro = 300

datam = cfl.read_cfl('./lung_mri/MRI_Raw_datam')
datam = datam[:,:,:,:,:num_ro,:]
cfl.write_cfl('./lung_mri/MRI_Raw_datam', datam)

In [2]:
dcf2m = cfl.read_cfl('./lung_mri/MRI_Raw_dcf2m')
dcf2m = dcf2m[:,:,:,:,:num_ro,:]
cfl.write_cfl('./lung_mri/MRI_Raw_dcf2m', dcf2m)

In [3]:
trajm = cfl.read_cfl('./lung_mri/MRI_Raw_trajm')
trajm = trajm[:,:,:,:,:num_ro,:]
cfl.write_cfl('./lung_mri/MRI_Raw_trajm', trajm)

In [4]:
# release RAM
#del datam
#del dcf2m
#del trajm

# iMoCo Reconstruction
step 1. XD-GRASP, motion-resolved reconstruction \
step 2. iMoCo reconstruction

In [5]:
!python imoco_recon/imoco_py/recon_xdgrasp.py ./lung_mri/MRI_Raw --device 0

tcmalloc: large alloc 1478942720 bytes == 0x3b96000 @  0x7fbe76ff11e7 0x7fbe74b175e1 0x7fbe74b8090d 0x7fbe74b81522 0x7fbe74c18bce 0x50a7f5 0x50cfd6 0x509918 0x50a64d 0x50c1f4 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7fbe76beeb97 0x5b26fa
Image Shape Estimate: [189, 125, 127]
running calibration
JsenseRecon: 100% 10/10 [00:09<00:00,  1.06it/s]
starting recon
running preconditioner
Time for preconditioner: 16.979700803756714 seconds.
running reconstruction
outer iter:0, res:1.0, time:25.231008768081665 seconds
outer iter:1, res:0.6153978109359741, time:24.001389026641846 seconds
outer iter:2, res:0.41544413566589355, time:23.29921007156372 seconds
outer iter:3, res:0.29002970457077026, time:22.87478804588318 seconds
outer iter:4, res:0.20256294310092926, time:23.207132577896118 seconds
outer iter:5, res:0.13752710819244385, time:22.94208288192749 seconds
outer iter:6, res:0.08738325536251068, time:23.270823001861572 seconds
outer iter:7, res:0.048247773200273514, 

In [22]:
!git clone https://github.com/ANTsX/ANTs.git
# make a build and install dir
!mkdir build install
%cd build
!cmake /content/ANTs

fatal: destination path 'ANTs' already exists and is not an empty directory.
/content/build
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Not using CCache
-- Setting build type to 'Release' as none was specified.
-- Performing Test C_HAS_WARNING-Wno-uninitialized
-- Performing Test C_HAS_WARNING-Wno-uninitialized - Success
-- Performing Test C_HAS_WARNING-Wno-unused-parameter
-- Performing Test C_HAS_WARNING-Wno-unused-parameter - Success
-- Per

In [ ]:
!make -j 2 2>&1

Streaming output truncated to the last 5000 lines.
-- Installing: /content/build/staging/include/ITK-5.1/itkLabelSelectionImageAdaptor.h
-- Installing: /content/build/staging/include/ITK-5.1/itkLabelImageGenericInterpolateImageFunction.hxx
-- Installing: /content/build/staging/include/ITK-5.1/itkLabelSelectionPixelAccessor.h
-- Installing: /content/build/staging/lib/cmake/ITK-5.1/Modules/GenericLabelInterpolator.cmake
-- Up-to-date: /content/build/staging/include/ITK-5.1
-- Installing: /content/build/staging/include/ITK-5.1/itkDenseFiniteDifferenceImageFilter.h
-- Installing: /content/build/staging/include/ITK-5.1/itkFiniteDifferenceFunction.hxx
-- Installing: /content/build/staging/include/ITK-5.1/itkFiniteDifferenceImageFilter.h
-- Installing: /content/build/staging/include/ITK-5.1/itkFiniteDifferenceSparseImageFilter.hxx
-- Installing: /content/build/staging/include/ITK-5.1/itkFiniteDifferenceFunction.h
-- Installing: /content/build/staging/include/ITK-5.1/itkFiniteDifferenceImageFi

In [2]:
# derive motion field 
!python imoco_recon/imoco_py/recon_imoco.py ./lung_mri/MRI_Raw --reg_flag 1 --device 0

tcmalloc: large alloc 1478942720 bytes == 0x4ba4000 @  0x7f02e7c9c1e7 0x7f02e57c25e1 0x7f02e582b90d 0x7f02e582c522 0x7f02e58c3bce 0x50a7f5 0x50cfd6 0x509918 0x50a64d 0x50c1f4 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7f02e7899b97 0x5b26fa
Calibration...
tcmalloc: large alloc 1474011136 bytes == 0x88f52000 @  0x7f02e7c9c1e7 0x7f02e57c25e1 0x7f02e5829420 0x7f02e581e7d5 0x7f02e58e9730 0x7f02e58b4317 0x567193 0x511341 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x588e91 0x59fe1e 0x7f02e58124ed 0x50a47f 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7f02e7899b97 0x5b26fa
JsenseRecon: 100% 10/10 [00:05<00:00,  1.72it/s]
Registration...
sh: 1: antsRegistration: not found
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/nibabel/loadsave.py", line 42, in load
    stat_result = os.stat(filename)
FileNotFoundError: [Errno 2] No such file or directory: './tmp_0Warp.nii.gz'

Durin

In [ ]:
# imoco recon
!python imoco_recon/imoco_py/recon_imoco.py ./lung_mri/MRI_Raw --device 0

# Display Image


In [ ]:
import sigpy.plot as pl
